In [1]:
import phasr as phr

In [2]:
phr.__version__

'0.5.4.dev17+g192a65555'

In [3]:
import numpy as np
pi=np.pi
import pandas as pd
import matplotlib.pyplot as plt

# Reference Nucleus and Charge Radius

In [4]:
# Reference nucleus for Al27 (taken from Noel and Hoferichter, 2024)

In [5]:
ai_27Al = [0.03125, 0.05727, 0.02301, -0.01565, -0.0163, -0.0039, 0.0010] 
R_27Al = 8.0 
nuc_ref_27Al = phr.nucleus('Al27_bestfit_example',Z=13,A=27,ai=ai_27Al,R=R_27Al) 

In [6]:
# According reference charge radius (also taken from Noel and Hoferichter, 2024)

In [7]:
r_ch_ref_27Al = 3.063 
dr_ch_ref_27Al = 0.031 
# 
r_ch_sq_ref_27Al = r_ch_ref_27Al**2 
dr_ch_sq_ref_27Al = 2*r_ch_ref_27Al*dr_ch_ref_27Al 
# 
r_ch_sq_ref_27Al, dr_ch_sq_ref_27Al 

(9.381969000000002, 0.18990600000000002)

# Import Datasets

In [8]:
# We use as an example ab-initio calculations for 27Al using the IMSRG over a collection of different input chiral Hamiltonians (courtesy of Matthias Heinz)

In [ ]:
AI_dict_27Al = phr.dirac_solvers.post_processing.correlation.prepare_results(Z=13,A=27,folder_path="./IMSRG_27Al/",name='Al27_ab_initio',r_cut=6)

Loaded datasets: ['1p82p0EM7p5', '1p82p0sim7p5', 'DN2LOGO', 'EM1p82p0', 'EM2p02p0', 'EM2p02p0PWA', 'EM2p22p0', 'N2LOsat', 'NIsample0090', 'NIsample0500', 'NIsample0606', 'NIsample0723', 'NIsample0750', 'NIsample0922', 'NIsample1141', 'NIsample1172', 'NIsample1173', 'NIsample1177', 'NIsample1469', 'NIsample1802', 'NIsample2007', 'NIsample2125', 'NIsample2245', 'NIsample2411', 'NIsample2460', 'NIsample3098', 'NIsample3105', 'NIsample3260', 'NIsample3319', 'NIsample3350', 'NIsample3448', 'NIsample3472', 'NIsample3480', 'NIsample3621', 'NIsample3895', 'NIsample4117']
data loaded from  ./tmp/nucleus_splines/charge_density_M0p_Al27_ab_initio_1p82p0EM7p5_x=0.0-6-0.05.txt
data loaded from  ./tmp/nucleus_splines/charge_density_laplace_M0p_Al27_ab_initio_1p82p0EM7p5_x=0.0-6.6000000000000005-0.05.txt
data loaded from  ./tmp/nucleus_splines/charge_density_M0n_Al27_ab_initio_1p82p0EM7p5_x=0.0-6-0.05.txt
data loaded from  ./tmp/nucleus_splines/charge_density_laplace_M0n_Al27_ab_initio_1p82p0EM7p5_x=

In [ ]:
# You may check for good behaviour of the distributions for large r and at the cutoff r_cut 
# You should choose the cutoff r_cut as large as possible while still reproducing the correct radii and not introducting any unphysical oscillations or behaviour in the high energy 
# If radii are consistent or inconsistent is listed as part of the prints above while the other thing can be checked from the plots

In [ ]:
r=np.arange(0,8+0.1,0.1)
for key in AI_dict_27Al:
    key_nucleus = AI_dict_27Al[key]['atom']
    plt.plot(r,key_nucleus.charge_density(r))
plt.title('Charge distributions for $^{27}$Al from IMSRG')
plt.xlabel(r'$r$ in fm')
plt.ylabel(r'$\rho_{\operatorname{ch}}$ in fm$^{-3}$')
plt.grid()
plt.yscale('log')
plt.xlim(0,8)
plt.show()

In [ ]:
# You may try to increase r_cut above and see how the high energy behaviour and warnings about radii consistentcy change (you should soon run into problems)

# Collect/Calculate basic quantities for correlation (radii, radii squared)

In [ ]:
# The basic function collects/calculates only radii and radii squared, if there are other quantities on file it will also load them

In [ ]:
AI_datasets_27Al = phr.dirac_solvers.post_processing.correlation.calculate_correlation_quantities(AI_dict_27Al)

# Correlation Radii

In [ ]:
# This is how you can fairly quickly and easily plot a correlation for quantities that are part of the datasets, using the example of radii squared

In [ ]:
fig,ax = plt.subplots(figsize=(5,4), dpi=200)
phr.dirac_solvers.post_processing.correlation.plot_correlation(ax,AI_datasets_27Al,x_str='rchsq',y_strs=['rpsq','rnsq','rwsq'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al,x_ref_label=r'$\langle r^2 \rangle^{\operatorname{ref}}_{\operatorname{ch}}$',y_str_label_trans=lambda x: r'$\langle r^2 \rangle_\operatorname{'+x[1]+'}$')
ax.annotate(r'$^{27}$Al', (0.97,0.96) ,xycoords='axes fraction' ,fontsize=10 ,horizontalalignment='right', verticalalignment='top',bbox=dict(boxstyle="round",facecolor=(1,1,1,0.75),edgecolor='lightgray'))
ax.set_ylabel(r'$\langle r^2 \rangle$ in fm$^{2}$')
ax.set_xlabel(r'$\langle r^2 \rangle_{\operatorname{ch}}$ in fm$^{2}$')
plt.tight_layout()
plt.show()

In [ ]:
# This is how you then calculate the implied values, propagating all uncertainties (the first parameters here are the same as for the plot)

In [ ]:
corr_dict_r2 = phr.dirac_solvers.post_processing.correlation.implications_of_correlation(AI_datasets_27Al,x_str='rchsq',y_strs=['rpsq','rnsq','rwsq'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al)

In [ ]:
# This is a special function for convenience for radii because we are usually interested in radii and not radii squared

In [ ]:
corr_dict_sqrsq = phr.dirac_solvers.post_processing.correlation.rsq_dict_add_r_vals(corr_dict_r2)

In [ ]:
# Just as an example, we may also correlate radii instead of radii squared 

In [ ]:
fig,ax = plt.subplots(figsize=(5,4), dpi=200)
phr.dirac_solvers.post_processing.correlation.plot_correlation(ax,AI_datasets_27Al,x_str='rch',y_strs=['rp','rn','rw'],x_ref=r_ch_ref_27Al,dx_ref=dr_ch_ref_27Al,x_ref_label=r'$\sqrt{\langle r^2 \rangle^{\operatorname{ref}}_{\operatorname{ch}}}$',y_str_label_trans=lambda x: r'$\sqrt{\langle r^2 \rangle_\operatorname{'+x[1]+'}}$')
ax.annotate(r'$^{27}$Al', (0.97,0.96) ,xycoords='axes fraction' ,fontsize=10 ,horizontalalignment='right', verticalalignment='top',bbox=dict(boxstyle="round",facecolor=(1,1,1,0.75),edgecolor='lightgray'))
ax.set_ylabel(r'$\sqrt{\langle r^2 \rangle}$ in fm$^{2}$')
ax.set_xlabel(r'$\sqrt{\langle r^2 \rangle_{\operatorname{ch}}}$ in fm$^{2}$')
plt.tight_layout()
plt.show()

In [ ]:
# We find the results are very consistent

In [ ]:
corr_dict_r = phr.dirac_solvers.post_processing.correlation.implications_of_correlation(AI_datasets_27Al,x_str='rch',y_strs=['rp','rn','rw'],x_ref=r_ch_ref_27Al,dx_ref=dr_ch_ref_27Al)

# Correlation for custom quantities

In [ ]:
# You may add the calculation of any quantity to the calculate_correlation_quantities call by adding it via the quantities_fct_dict

In [ ]:
# As an example we want to add the r^4 moments of the densities

In [ ]:
from scipy.integrate import quad
def r4_ch_moment_example_function(individual_nucleus):
    norm = individual_nucleus.total_charge
    r4 = 4*pi/norm * quad(lambda x: x**2*x**4*individual_nucleus.charge_density(x), 0 , np.inf ,limit=1000)[0]
    return r4
def r4_p_moment_example_function(individual_nucleus):
    norm = individual_nucleus.Z
    r4 = 4*pi/norm * quad(lambda x: x**2*x**4*individual_nucleus.proton_density(x), 0 , np.inf ,limit=1000)[0]
    return r4
def r4_n_moment_example_function(individual_nucleus):
    norm = individual_nucleus.A - individual_nucleus.Z
    r4 = 4*pi/norm * quad(lambda x: x**2*x**4*individual_nucleus.neutron_density(x), 0 , np.inf ,limit=1000)[0]
    return r4
def r4_w_moment_example_function(individual_nucleus):
    norm = individual_nucleus.weak_charge
    r4 = 4*pi/norm * quad(lambda x: x**2*x**4*individual_nucleus.weak_density(x), 0 , np.inf ,limit=1000)[0]
    return r4

# collect all functions in the dictionary, the keys are the names used to identify the quantities (in case you have other parameters/inputs you want them represented in the key)
r4_moments_fct_dict={'r4ch':r4_ch_moment_example_function,'r4p':r4_p_moment_example_function,'r4n':r4_n_moment_example_function,'r4w':r4_w_moment_example_function}

In [ ]:
# By adding this dictionary to the call these functions are also all evalualted for all sample nuclei

In [ ]:
AI_datasets_27Al = phr.dirac_solvers.post_processing.correlation.calculate_correlation_quantities(AI_dict_27Al,quantities_fct_dict=r4_moments_fct_dict)

In [ ]:
# And can be accessed now by the keys of the dictionary 

In [ ]:
fig,ax = plt.subplots(figsize=(5,4), dpi=200)
phr.dirac_solvers.post_processing.correlation.plot_correlation(ax,AI_datasets_27Al,x_str='rchsq',y_strs=['r4ch','r4p','r4n','r4w'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al,x_ref_label=r'$\langle r^2 \rangle^{\operatorname{ref}}_{\operatorname{ch}}$',y_str_label_trans=lambda x: r'$\langle r^4 \rangle_{'+x[2:]+'}$')
ax.annotate(r'$^{27}$Al', (0.97,0.96) ,xycoords='axes fraction' ,fontsize=10 ,horizontalalignment='right', verticalalignment='top',bbox=dict(boxstyle="round",facecolor=(1,1,1,0.75),edgecolor='lightgray'))
ax.set_ylabel(r'$\langle r^4 \rangle$ in fm$^{4}$')
ax.set_xlabel(r'$\langle r^2 \rangle_{\operatorname{ch}}$ in fm$^{2}$')
plt.tight_layout()
plt.show()

In [ ]:
# We again observe a correlation, which however is a bit weaker than for the radii. The plot shows that the microscopic constraintes from the chiral Hamiltonians seem to change r^2 moments and r^4 moments in a correlated way. 

In [ ]:
# We may also predict expected values for the r^4 moments based on this correlation

In [ ]:
corr_dict_r = phr.dirac_solvers.post_processing.correlation.implications_of_correlation(AI_datasets_27Al,x_str='rchsq',y_strs=['r4ch','r4p','r4n','r4w'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al)

In [ ]:
# We may calulate the r4 moment also for the reference nucleus and compare to our correlation result for the r^4 moment from the charge distribution
r4_ch_ref = r4_ch_moment_example_function(nuc_ref_27Al) 
r4_ch_ref 

In [ ]:
# Which is consistent within uncertainty (even without taking the error on r^4 from the reference nucleus into account, which we did not calculate here)

# Correlation for implemented Quantities

In [ ]:
# For some (more or less common) quantities we have implemented functions to simplify the process of calculating these for all samples

## Leading Overlap Integrals in $\mu \to e$ conversion

In [ ]:
# Correlation with fixed charge distribution for the solutions of the muon and electron wave functions

In [ ]:
AI_datasets_27Al = phr.dirac_solvers.post_processing.correlation.calculate_correlation_SI_overlap_integrals(AI_datasets_27Al,reference_nucleus=nuc_ref_27Al)

In [ ]:
fig,ax = plt.subplots(figsize=(5,4), dpi=200)
phr.dirac_solvers.post_processing.correlation.plot_correlation(ax,AI_datasets_27Al,x_str='rchsq',y_strs=['Sp_rhoch_Al27_bestfit_example','Sn_rhoch_Al27_bestfit_example','Vp_rhoch_Al27_bestfit_example','Vn_rhoch_Al27_bestfit_example'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al,x_ref_label=r'$\langle r^2 \rangle^{\operatorname{ref}}_{\operatorname{ch}}$',y_str_label_trans=lambda x: r'$'+x[0]+'_'+x[1]+'$')
ax.annotate(r'$^{27}$Al', (0.97,0.96) ,xycoords='axes fraction' ,fontsize=10 ,horizontalalignment='right', verticalalignment='top',bbox=dict(boxstyle="round",facecolor=(1,1,1,0.75),edgecolor='lightgray'))
ax.set_ylabel(r'overlap integral in $m_{\mu}^{5/2}$')
ax.set_xlabel(r'$\langle r^2 \rangle_{\operatorname{ch}}$ in fm$^{2}$')
plt.tight_layout()
plt.show()

In [ ]:
corr_dict_ov1 = phr.dirac_solvers.post_processing.correlation.implications_of_correlation(AI_datasets_27Al,x_str='rchsq',y_strs=['Sp_rhoch_Al27_bestfit_example','Sn_rhoch_Al27_bestfit_example','Vp_rhoch_Al27_bestfit_example','Vn_rhoch_Al27_bestfit_example'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al)

In [ ]:
# Correlation with the charge distribution constructed from the ab initio calculation also for the solutions of the muon and electron wave functions

In [ ]:
AI_datasets_27Al = phr.dirac_solvers.post_processing.correlation.calculate_correlation_SI_overlap_integrals(AI_datasets_27Al)

In [ ]:
fig,ax = plt.subplots(figsize=(5,4), dpi=200)
phr.dirac_solvers.post_processing.correlation.plot_correlation(ax,AI_datasets_27Al,x_str='rchsq',y_strs=['Sp_rhoch_from_dataset','Sn_rhoch_from_dataset','Vp_rhoch_from_dataset','Vn_rhoch_from_dataset'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al,x_ref_label=r'$\langle r^2 \rangle^{\operatorname{ref}}_{\operatorname{ch}}$',y_str_label_trans=lambda x: r'$'+x[0]+'_'+x[1]+'$')
ax.annotate(r'$^{27}$Al', (0.97,0.96) ,xycoords='axes fraction' ,fontsize=10 ,horizontalalignment='right', verticalalignment='top',bbox=dict(boxstyle="round",facecolor=(1,1,1,0.75),edgecolor='lightgray'))
ax.set_ylabel(r'overlap integral in $m_{\mu}^{5/2}$')
ax.set_xlabel(r'$\langle r^2 \rangle_{\operatorname{ch}}$ in fm$^{2}$')
plt.tight_layout()
plt.show()

In [ ]:
corr_dict_ov1 = phr.dirac_solvers.post_processing.correlation.implications_of_correlation(AI_datasets_27Al,x_str='rchsq',y_strs=['Sp_rhoch_from_dataset','Sn_rhoch_from_dataset','Vp_rhoch_from_dataset','Vn_rhoch_from_dataset'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al)

## Charge and Weak form factor

In [ ]:
# Correlation for the form factor evaluated at a specific value of momentum transfer

In [ ]:
q_Qweak=0.77802*phr.constants.hc
q_Qweak # MeV

In [ ]:
AI_datasets_27Al = phr.dirac_solvers.post_processing.correlation.calculate_correlation_form_factors(AI_datasets_27Al,q_exp=q_Qweak)

In [ ]:
fig,ax = plt.subplots(figsize=(5,4), dpi=200)
phr.dirac_solvers.post_processing.correlation.plot_correlation(ax,AI_datasets_27Al,x_str='Fch_q153.524',y_strs=['Fw_q153.524'],x_ref=nuc_ref_27Al.form_factor(q_Qweak),x_ref_label=r'$F^\operatorname{ref}_\operatorname{ch}(q_\operatorname{exp})$',y_str_label_trans=lambda x: r'$'+x[0]+'_'+x[1]+'$',plot_color_legend=False)
ax.annotate(r'$^{27}$Al', (0.97,0.96) ,xycoords='axes fraction' ,fontsize=10 ,horizontalalignment='right', verticalalignment='top',bbox=dict(boxstyle="round",facecolor=(1,1,1,0.75),edgecolor='lightgray'))
ax.set_ylabel(r'$F_\operatorname{w}(q_\operatorname{exp})$')
ax.set_xlabel(r'$F_\operatorname{ch}(q_\operatorname{exp})$')
plt.tight_layout()
plt.show()

## Parity Violating Asymmetry in elastic electron scattering

In [ ]:
# Correlation for the parity violating asymmetry / left right asymmetry of elastic electron nucleus scattering 

In [ ]:
# Considered kinematics

In [ ]:
E_beam_Qweak = 1157 
angle_Qweak = 7.61 * pi/180 

In [ ]:
# One may either pass a fixed energy and angle (as in the example herer) or a fixed energy, and an angle range and according (unnormalized) acceptances. In the second case the result is averaged over the angles and the acceptance with the equation as quoted in the literature (see e.g. sub. material in CREX & PREX)

In [ ]:
AI_datasets_27Al = phr.dirac_solvers.post_processing.correlation.calculate_correlation_left_right_asymmetry(AI_datasets_27Al,E_exp=E_beam_Qweak,theta_exp=angle_Qweak,left_right_asymmetry_args={'save_and_load_phase_shifts': True})

In [ ]:
AI_datasets_27Al = phr.dirac_solvers.post_processing.correlation.calculate_correlation_left_right_asymmetry(AI_datasets_27Al,reference_nucleus=nuc_ref_27Al,E_exp=E_beam_Qweak,theta_exp=angle_Qweak,left_right_asymmetry_args={'save_and_load_phase_shifts': True})

In [ ]:
# Experimental results from the Qweak collaboration as comparison
APV_exp=2.16*1e-6
dAPV_exp=np.sqrt(0.11**2+0.16**2)*1e-6

In [ ]:
fig,ax = plt.subplots(figsize=(5,4), dpi=200)
phr.dirac_solvers.post_processing.correlation.plot_correlation(ax,AI_datasets_27Al,x_str='rchsq',y_strs=['APV_E1157.00_theta0.1328_rhoch_Al27_bestfit_example','APV_E1157.00_theta0.1328_rhoch_from_dataset'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al,
                                                               x_ref_label=r'$\langle r^2 \rangle^{\operatorname{ref}}_{\operatorname{ch}}$',y_str_label_trans=lambda x: x[:3]+('2' if x[-12:]=='from_dataset' else '1'))
ax.annotate(r'$^{27}$Al', (0.97,0.96) ,xycoords='axes fraction' ,fontsize=10 ,horizontalalignment='right', verticalalignment='top',bbox=dict(boxstyle="round",facecolor=(1,1,1,0.75),edgecolor='lightgray'))

xmin, xmax = ax.get_xlim()
ax.plot([xmin,xmax],[APV_exp,APV_exp],linestyle='--',color='C4',zorder=-3)
ax.fill_between([xmin,xmax],2*[APV_exp-dAPV_exp],2*[APV_exp+dAPV_exp],alpha=0.25,color='C4',zorder=-4)
ax.annotate(r'$Q_{\operatorname{weak}}$', (xmin+1e-2,APV_exp-1.05*dAPV_exp),horizontalalignment='left', verticalalignment='top',color='C4')

ax.set_ylabel(r'APV')
ax.set_xlabel(r'$\langle r^2 \rangle_{\operatorname{ch}}$ in fm$^{2}$')
plt.tight_layout()
plt.show()

In [ ]:
corr_dict_APV = phr.dirac_solvers.post_processing.correlation.implications_of_correlation(AI_datasets_27Al,x_str='rchsq',y_strs=['APV_E1157.00_theta0.1328_rhoch_Al27_bestfit_example','APV_E1157.00_theta0.1328_rhoch_from_dataset'],x_ref=r_ch_sq_ref_27Al,dx_ref=dr_ch_sq_ref_27Al)

In [ ]:
# Reverse Correlation: Deducing the nucleus skin based on the APV measurement 

In [ ]:
fig,ax = plt.subplots(figsize=(5,4), dpi=200)
phr.dirac_solvers.post_processing.correlation.plot_correlation(ax,AI_datasets_27Al,x_str='APV_E1157.00_theta0.1328_rhoch_from_dataset',y_strs=['rw-rch','rn-rp'],x_ref=APV_exp,dx_ref=dAPV_exp,
                                                               x_ref_label=r'$Q_{\operatorname{weak}}$',y_str_label_trans=lambda x: x)
ax.annotate(r'$^{27}$Al', (0.97,0.96) ,xycoords='axes fraction' ,fontsize=10 ,horizontalalignment='right', verticalalignment='top',bbox=dict(boxstyle="round",facecolor=(1,1,1,0.75),edgecolor='lightgray'))
ax.set_ylabel(r'skin thickness in fm')
ax.set_xlabel(r'APV')
plt.tight_layout()
plt.show()

In [ ]:
corr_dict_rskin = phr.dirac_solvers.post_processing.correlation.implications_of_correlation(AI_datasets_27Al,x_str='APV_E1157.00_theta0.1328_rhoch_from_dataset',y_strs=['rw-rch','rn-rp'],x_ref=APV_exp,dx_ref=dAPV_exp)

In [ ]:
# Very large uncertainty due to the large uncertainty on the experimental measurement